# Confirm that noun banks are accessible by ipykernal.
## Shared Pandas dfs are transported as pickle files.
## Read using the %store -r '...' 'magic method'

In [2]:
# get noun banks
%store -r all_usa_states
%store -r all_world_countries
%store -r all_elements
%store -r many_animals
%store -r many_greek_gods
%store -r many_roman_gods
%store -r many_cosmetic_items
%store -r many_office_supplies


## Delete old database

In [3]:
import os
#delete old database
os.remove('NounBankSQlite.db')



# Creating an engine will store a file "NounBankSQlite.db" at SQlite-Databases/NounBankSQlite.db
#### 

In [4]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine

DATABASE_LOCATION = "sqlite:///NounBankSQlite.db"

    
def post_to_sqlite():
    # establish database connection
    engine = create_engine(DATABASE_LOCATION)
    # populate database with noun banks
    all_usa_states.to_sql('all_usa_states', engine, if_exists='replace')
    all_world_countries.to_sql('all_world_countries', engine, if_exists='replace')
    all_elements.to_sql('all_elements', engine, if_exists='replace')
    many_animals.to_sql('many_animals', engine, if_exists='replace')
    many_greek_gods.to_sql('many_greek_gods', engine, if_exists='replace')
    many_roman_gods.to_sql('many_roman_gods', engine, if_exists='replace')
    many_cosmetic_items.to_sql('many_cosmetic_items', engine, if_exists='replace')
    many_office_supplies.to_sql('many_office_supplies', engine, if_exists='replace')
    # close database connection
    

post_to_sqlite()

## Confirm that future updates to the Noun Bank do not raise any assertion errors

In [5]:
import unittest as ut

class TestPostToSqlite(ut.TestCase):
    def __init__(self, *args, **kwargs):
        self.engine = create_engine(DATABASE_LOCATION) # create an engine to connect to the database for testing
        super(TestPostToSqlite, self).__init__(*args, **kwargs) # call the parent class's constructor

    def test_connecting_to_sqlite(self):
        self.assertEqual(self.engine.name, 'sqlite') # check that the engine is connected to a sqlite database
        
    def test_states_start_with_new(self):
        df = pd.read_sql('SELECT all_usa_states FROM all_usa_states WHERE all_usa_states LIKE "New%"', self.engine) # select all states that start with "New"
        self.assertEqual(df.values.tolist(), [['New Hampshire'], ['New Jersey'], ['New Mexico'], ['New York']]) # check that the correct states are returned
        
    def test_longest_len_greek_god(self):
        df = pd.read_sql('SELECT MAX(LENGTH(many_greek_gods)), many_greek_gods FROM many_greek_gods', self.engine)
        self.assertEqual(df['many_greek_gods'][0], 'hephaestus' ) # check that the greek god with the longest name is Hephaestus
        self.assertEqual(df['MAX(LENGTH(many_greek_gods))'][0], 10) # check that the length of the longest name is 11

    def test_element_with_largest_atomic_number(self):
        df = pd.read_sql('SELECT all_elements, AtomicNumber FROM all_elements WHERE AtomicNumber = (SELECT MAX(AtomicNumber) FROM all_elements)', self.engine)
        self.assertEqual(df['all_elements'][0], 'Oganesson') # check that the element with the largest atomic number is Oganesson
        self.assertEqual(df['AtomicNumber'][0], 118) # check that the atomic number of Oganesson is 118


#run the tests

if __name__ == '__main__':
    ut.main(argv=[''], verbosity=4, exit=False)

test_connecting_to_sqlite (__main__.TestPostToSqlite) ... ok
test_element_with_largest_atomic_number (__main__.TestPostToSqlite) ... ok
test_longest_len_greek_god (__main__.TestPostToSqlite) ... ok
test_states_start_with_new (__main__.TestPostToSqlite) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.021s

OK


In [18]:
import numpy as np


class Question:
    def __init__(self) -> None:
        self.engine = create_engine(DATABASE_LOCATION)
        self.available_tables = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', self.engine)
        print(self.available_tables)
        self.chosen_table = np.random.choice(self.available_tables['name'])

        if self.chosen_table == 'all_usa_states':
            self.NOUN = "A State in the USA"
        elif self.chosen_table == 'all_world_countries':
            self.NOUN = "A Country in the World"
        elif self.chosen_table == 'all_elements':
            self.NOUN = "An Element"
        elif self.chosen_table == 'many_animals':
            self.NOUN = "An Animal"
        elif self.chosen_table == 'many_greek_gods':
            self.NOUN = "A Greek God"
        elif self.chosen_table == 'many_roman_gods':
            self.NOUN = "A Roman God"
        elif self.chosen_table == 'many_cosmetic_items':
            self.NOUN = "A Cosmetic Item"
        elif self.chosen_table == 'many_office_supplies':
            self.NOUN = "An Office Supply"

        self.question_filters = ["Begins With The Letter", "Ends With The Letter"]

    def get_question(self):
        # print(self.options)
        self.chosen_filter = np.random.choice(self.question_filters)
        # make a list of all the first letters of the nouns in the chosen table
        if self.chosen_filter == "Begins With The Letter":
            # make a set called first letters that contains all the first letters of the nouns in the chosen table
            first_letters = pd.read_sql('SELECT SUBSTR(' + self.chosen_table + ', 1, 1) FROM ' + self.chosen_table, self.engine)
            # make one dimensional array of first letters
            first_letters = first_letters.values.tolist()
            # make first letters into a list rather than a list of lists
            first_letters = [item for sublist in first_letters for item in sublist]
            self.random_letter = np.random.choice(first_letters)
            # print(first_letters)
            
        if self.chosen_filter == "Ends With The Letter":
            # make a set called last letters that contains all the last letters of the nouns in the chosen table
            last_letters = pd.read_sql('SELECT SUBSTR(' + self.chosen_table + ', -1, 1) FROM ' + self.chosen_table, self.engine)
            # make one dimensional array of first letters
            last_letters = last_letters.values.tolist()
            # make first letters into a list rather than a list of lists
            last_letters = [item for sublist in last_letters for item in sublist]
            self.random_letter = np.random.choice(last_letters)
            # print(last_letters)
        
        
        return "Name " + self.NOUN + " That " + self.chosen_filter + " " + self.random_letter.capitalize() + "."

    def check_answer(self, potential_answer):
        if self.chosen_filter == "Begins With The Letter":
            if potential_answer[0].lower() == self.random_letter.lower():
                # check if potential answer is in the chosen table in the database
                all_nouns_in_table = pd.read_sql('SELECT ' + self.chosen_table + ' FROM ' + self.chosen_table, self.engine)
                # print(all_nouns_in_table.values.tolist())
                all_nouns_in_table = [item for sublist in all_nouns_in_table.values.tolist() for item in sublist]
                # make all nouns in table lowercase
                all_nouns_in_table = [x.lower() for x in all_nouns_in_table]
                # check if potential answer is in the list of all nouns in the table
                if potential_answer.lower() in all_nouns_in_table:
                    print("Correct!")
                    print(potential_answer.lower().capitalize() + " is a " + self.NOUN + " that begins with the letter " + self.random_letter + ".")
                else:
                    print("That is not a " + self.NOUN + "!")
            else:
                print("That does not begin with the letter " + self.random_letter + "!")
                
        if self.chosen_filter == "Ends With The Letter":
            if potential_answer[-1].lower() == self.random_letter.lower():
                # check if potential answer is in the chosen table in the database
                all_nouns_in_table = pd.read_sql('SELECT ' + self.chosen_table + ' FROM ' + self.chosen_table, self.engine)
                # print(all_nouns_in_table.values.tolist())
                all_nouns_in_table = [item for sublist in all_nouns_in_table.values.tolist() for item in sublist]
                # make all nouns in table lowercase
                all_nouns_in_table = [x.lower() for x in all_nouns_in_table]
                # check if potential answer is in the list of all nouns in the table
                if potential_answer.lower() in all_nouns_in_table:
                    print("CORRECT!")
                    print(potential_answer.lower().capitalize() + " is a " + self.NOUN + " that ends with the letter " + self.random_letter + ".")
                else:
                    print("That is not a " + self.NOUN + "!")
            else:
                print("That does not end with the letter " + self.random_letter + "!")

In [25]:
q = Question()
print(q.get_question())

                   name
0        all_usa_states
1   all_world_countries
2          all_elements
3          many_animals
4       many_greek_gods
5       many_roman_gods
6   many_cosmetic_items
7  many_office_supplies
Name A Greek God That Begins With The Letter A.


In [28]:
q.check_answer("Aphrodite")


That is not a A Greek God!
